In [1]:
!pip install dlib

    100% |████████████████████████████████| 3.1MB 2.6MB/s ta 0:00:011
  Running setup.py bdist_wheel for dlib ... done
  Stored in directory: /home/allen/.cache/pip/wheels/ab/e9/6b/5206171e6a24d8cb6653936ccfd656b7d6f6c631c1b5f93970
Successfully built dlib
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
import dlib
import cv2
from PIL import Image 
def to_pil(cv_img):
    img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
    return Image.fromarray(img)

detector = dlib.get_frontal_face_detector()

In [31]:
def get_bbox(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    dets = detector(img, 1)
    bboxes = []
    for i in range(len(dets)):
        tl = dets[i].tl_corner()
        br = dets[i].br_corner()
        x1, y1 = tl.x, tl.y
        x2, y2 = br.x, br.y
        bboxes.append((x1, y1, x2, y2))
    return bboxes

In [32]:
img = cv2.imread('/home/allen/two_people.jpg')
bboxes = get_bbox(img)

In [33]:
for box in bboxes:
    x1, y1, x2, y2 = box
    img = cv2.rectangle(img, (x1, y1), (x2, y2), (255,255,255), 2)

In [3]:
pip install psutil

    100% |████████████████████████████████| 450kB 3.5MB/s ta 0:00:011
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /home/allen/.cache/pip/wheels/52/41/b0/bf50409fe2b1d3b79afa3eed71b54b3e30fe5b695db2c7ba2e
Successfully built psutil
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import ray
import cv2
import time

# Start Ray.
ray.init()

@ray.remote
def f(x):
    time.sleep(1)
    return x

In [ ]:
def detection(img):
    h, w = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    det_start = time.time()
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    det_time = time.time() - det_start
    render_start = time.time()
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    render_time = time.time() - render_start
    
    cv2.putText(img, 'Size   : {}x{}'.format(w,h)              , (10,30), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255,255,255), 4)
    cv2.putText(img, 'Render : {:.2f}ms'.format(render_time*1000), (10,50), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255,255,255), 4)
    cv2.putText(img, 'Det    : {:.2f}ms'.format(det_time*1000)   , (10,70), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255,255,255), 4)
    cv2.putText(img, 'Size   : {}x{}'.format(w,h)              , (10,30), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,0,0), 2)
    cv2.putText(img, 'Render : {:.2f}ms'.format(render_time*1000), (10,50), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,0,0), 2)
    cv2.putText(img, 'Det    : {:.2f}ms'.format(det_time*1000)   , (10,70), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,0,0), 2)
    
    return img

In [ ]:
import ray
import cv2
import time

# Start Ray.
ray.init()

@ray.remote
def detection(img):
    h, w = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    det_start = time.time()
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    det_time = time.time() - det_start
    render_start = time.time()
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    render_time = time.time() - render_start
    
    cv2.putText(img, 'Size   : {}x{}'.format(w,h)              , (10,30), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255,255,255), 4)
    cv2.putText(img, 'Render : {:.2f}ms'.format(render_time*1000), (10,50), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255,255,255), 4)
    cv2.putText(img, 'Det    : {:.2f}ms'.format(det_time*1000)   , (10,70), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255,255,255), 4)
    cv2.putText(img, 'Size   : {}x{}'.format(w,h)              , (10,30), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,0,0), 2)
    cv2.putText(img, 'Render : {:.2f}ms'.format(render_time*1000), (10,50), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,0,0), 2)
    cv2.putText(img, 'Det    : {:.2f}ms'.format(det_time*1000)   , (10,70), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,0,0), 2)
    
    return img

# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# To capture video from webcam. 
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')

while True:
    # Read the frame
    _, img = cap.read()
    
    r_img = None
    
    render_img = detection.remote(img)
    
    r_img = ray.get(render_img)
    
    # Display
    if r_img is not None:
        cv2.imshow('img', r_img)
    else:
        cv2.imshow('img', img)

    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
        
# Release the VideoCapture object
cap.release()